# Space-Time Cube

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('../data/transaction/v1.xlsx')
df.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan,perbedaan_jarak
0,16029,8.132182e+10,purwaramadhika@gmail.com,2023-05-04 00:00:00,Kurir Barang,2023-05-04 05:06:00,"komp gading tutuka 2 blok k8 no 8, Cingcin, Ke...",-7.021726,107.505362,"Jl. Gading Tutuka 2 No.42, Ciluncat, Kec. Cang...",-7.043931,107.540715,2023-05-04 05:31:00,2.50,8800,100.0,Dompet + Tunai,7.490,4.990
1,27243,NaN,deerah60@gmail.com,2023-05-08 00:00:00,Food,2023-05-08 11:23:00,Jl. Dakota 28 Sukaraja Cicendo Bandung,-6.894789,107.569360,"No.154. Jalan Papajaran, Husen Sastranegara",-6.904878,107.579212,2023-05-08 11:51:00,1.89,7200,150.0,Dompet,6.880,4.990
2,14783,8.129765e+10,mrayfan17@gmail.com,2023-05-11 00:00:00,Food,2023-05-11 19:06:00,"Jl. Terusan Jakarta No.6, Babakan Surabaya, Ke...",-6.913613,107.645752,"Jl. Soekarno Hatta, Babakan Penghulu, Bandung ...",-6.937207,107.690535,2023-05-11 19:26:00,4.71,12000,150.0,Tunai,9.698,4.988
3,26411,NaN,daddykurniadi4647@gmail.com,2023-05-02 00:00:00,Food,2023-05-02 19:24:00,GF- Outdoor JL Ciumbuleuit No 147. He...,-6.875969,107.604215,"Jl. Cipaku Indah X No.39, Ledeng, Kec...",-6.859268,107.600874,2023-05-02 19:37:00,2.36,8800,150.0,Dompet,7.347,4.987
4,16842,8.122494e+10,amrytaufikblur@gmail.com,2023-05-14 00:00:00,Ride,2023-05-14 08:00:00,"jl dipatiukur no 107, lebakgede",-6.885659,107.614298,4J4F+4GM jalan gagak dalam,-6.931904,107.627052,2023-05-14 08:09:00,1.99,8000,150.0,Dompet,6.972,4.982


In [7]:
# Convert 'tanggal' to datetime format
df['datetime_tugas'] = pd.to_datetime(df['datetime_tugas'])
df['tanggal'] = pd.to_datetime(df['tanggal'])

# Filter data for one specific day (e.g., 2023-05-04)
desired_date = '2023-05-14'
filtered_df = df[df['tanggal'].dt.date == pd.to_datetime(desired_date).date()]


# Create the 3D visualization
fig = go.Figure()

for index, row in filtered_df.iterrows():
    fig.add_trace(go.Scatter3d(
        x=[row['long_jemput']],
        y=[row['lat_jemput']],
        z=[row['datetime_tugas']],
        mode='lines+markers',
        line=dict(color='blue'),
        marker=dict(size=5),
        text=['Pickup', 'Dropoff'],
    ))

# Customize the layout
fig.update_layout(
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Time',
        xaxis=dict(range=[107.5, 107.7]),
        yaxis=dict(range=[-7.1, -6.8]),
        zaxis=dict(range=[filtered_df['datetime_tugas'].min(), filtered_df['datetime_tugas'].max()]),
    ),
    title='Space-Time Path 3D Visualization',
)



# Save the plot as an HTML file
fig.write_html('space_time_path_visualization.html')


In [10]:
# Convert 'tanggal' to datetime format
df['datetime_tugas'] = pd.to_datetime(df['datetime_tugas'])

# Extract the hour component from the timestamp
df['hour'] = df['datetime_tugas'].dt.hour

# Aggregate the data for 24 hours by taking the mean
df_grouped = df.groupby('hour').agg({
    'lat_jemput': 'mean',
    'long_jemput': 'mean',
    'lat_tujuan': 'mean',
    'long_tujuan': 'mean',
}).reset_index()

# Create the 3D visualization
fig = go.Figure()

for index, row in df_grouped.iterrows():
    fig.add_trace(go.Scatter3d(
        x=[row['long_jemput'], row['long_tujuan']],
        y=[row['lat_jemput'], row['lat_tujuan']],
        z=[row['hour'], row['hour']],
        mode='lines+markers',
        line=dict(color='blue'),
        marker=dict(size=5),
        text=['Pickup', 'Dropoff'],
    ))

# Customize the layout
fig.update_layout(
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Hour of the Day',
        xaxis=dict(range=[107.5, 107.7]),
        yaxis=dict(range=[-7.1, -6.8]),
    ),
    title='Space-Time Path 3D Visualization (24-Hours Aggregated)',
)


# Save the plot as an HTML file
fig.write_html('space_time_path_visualization2.html')